# Using Donut by Hugging Face

In [ ]:
!pip install transformers pytorch-lightning pdf2image pypdf

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch
from pdf2image import convert_from_path
from PIL import Image
import os

In [ ]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "at

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0): DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )

In [ ]:
# Install Poppler (Required for pdf2image)
!apt-get install -y poppler-utils

from pdf2image import convert_from_path
from PIL import Image

# pdf_path = "/content/Screenshot 2025-02-14 115920.png"  # Replace with your actual PDF file
# # images = convert_from_path(pdf_path)
# images = "/content"

# # Save images from PDF
# image_paths = []
# for i, image in enumerate(images):
#     img_path = f"page_{i}.png"
#     image.save(img_path, "PNG")
#     image_paths.append(img_path)

# print("PDF successfully converted to images:", image_paths)

from PIL import Image

# Use the existing image directly
image_paths = ["/content/sample_data/Screenshot 2025-02-14 115920.png"]  # Add image paths here

print("Using images directly:", image_paths)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Using images directly: ['/content/sample_data/Screenshot 2025-02-14 115920.png']


In [ ]:
# from PIL import Image
# import torch

def read_pdf_image(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

    # Get decoder start token ID
    decoder_start_token_id = model.config.decoder_start_token_id or model.config.bos_token_id or 0
    decoder_input_ids = torch.tensor([[decoder_start_token_id]]).to(device)

    # Generate text
    outputs = model.generate(pixel_values, decoder_input_ids=decoder_input_ids, max_length=512)
    generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    return generated_text

# Save extracted text to a file
output_file = "extracted_text.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for img_path in image_paths:
        print(f"Reading: {img_path}")
        extracted_text = read_pdf_image(img_path)
        f.write(f"=== Extracted Text from {img_path} ===\n")
        f.write(extracted_text + "\n\n")  # Add spacing between pages

print(f"Extracted text saved to {output_file}")

Legacy behavior is being used. The current behavior will be deprecated in version 5.0.0. In the new behavior, if both images and text are provided, the default value of `add_special_tokens` will be changed to `False` when calling the tokenizer if `add_special_tokens` is unset. To test the new behavior, set `legacy=False`as a processor call argument.


Reading: /content/sample_data/Screenshot 2025-02-14 115920.png
Extracted text saved to extracted_text.txt


# Using TrOCR by Hugging face

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

# Load TrOCR model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# Load image from local file path
image_path = "/content/sample_data/Screenshot 2025-02-14 115920.png"  # Replace with the correct path
image = Image.open(image_path).convert("RGB")

# Process the image
pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

# Decode the text
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Extracted Text:", generated_text)


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.48.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

Extracted Text: 0 1


# Using PaliGemma 3b-pt by Hugging Face

In [ ]:
# Install vLLM from pip:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45

In [ ]:
# Load and run the model:
!vllm serve "google/paligemma-3b-pt-224"

2025-02-17 08:26:07.187945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739780767.209283   42934 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739780767.215291   42934 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 02-17 08:26:12 __init__.py:194] No platform detected, vLLM is running on UnspecifiedPlatform
INFO 02-17 08:26:16 api_server.py:840] vLLM API server version 0.7.2
INFO 02-17 08:26:16 api_server.py:841] args: Namespace(subparser='serve', model_tag='google/paligemma-3b-pt-224', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api

In [ ]:
from transformers import pipeline
from PIL import Image

# Load the image-text-to-text model
pipe = pipeline(
    "image-to-text",
    model="google/paligemma-3b-pt-224",
    model_kwargs={"torch_dtype": "auto"},
    device=-1  # Use CPU
)

# Load an image correctly as a PIL image
image_path = "/content/sample_data/Screenshot 2025-02-14 115920.png"  # Ensure this file exists
image = Image.open(image_path).convert("RGB")  # Convert to RGB mode

# Define a text prompt (optional)
prompt = "Describe this image in detail."

# Pass the image and text correctly within a dictionary
# The 'image' key should have the PIL image, and the 'text' key should have the prompt
output = pipe({"image": image, "text": prompt}, max_new_tokens=100)

# Print the result
print(output[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


TypeError: Incorrect format used for image. Should be an url linking to an image, a base64 string, a local path, or a PIL image.

# Using Groq Llama-3.2-11b-vision-preview

In [6]:
!pip install groq PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [8]:
# from groq import Groq
# import base64
# import requests
# import os

# # Set your Groq API key
# os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

# # Initialize the Groq client
# client = Groq()

# def analyze_image(image_path, text_prompt):
#     # Read and encode the image in base64
#     with open(image_path, "rb") as image_file:
#         encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

#     # Define the payload
#     messages = [
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": text_prompt},
#                 {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
#             ]
#         }
#     ]

#     # Call the Groq API
#     completion = client.chat.completions.create(
#         model="llama-3.2-11b-vision-preview",
#         messages=messages,
#         temperature=1,
#         max_completion_tokens=1024,
#         top_p=1,
#         stream=False,
#         stop=None,
#     )

#     # Print the result
#     return completion.choices[0].message.content

# # Example usage
# if __name__ == "__main__":
#     image_path = "/content/sample_data/Screenshot 2025-02-17 192841.png"  # Path to your image
#     text_prompt = "Analyze this image and provide insights."
#     result = analyze_image(image_path, text_prompt)
#     print("AI Analysis:", result)


from groq import Groq
import base64
import requests
from PyPDF2 import PdfReader

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

# Initialize the Groq client
client = Groq()

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text if text else "No readable text found."

def analyze_file(file_path, text_prompt):
    if file_path.lower().endswith(".pdf"):
        extracted_text = extract_text_from_pdf(file_path)
        content = [{"type": "text", "text": f"{text_prompt}\nExtracted Text:\n{extracted_text}"}]
    else:
        encoded_image = encode_image(file_path)
        content = [
            {"type": "text", "text": text_prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
        ]

    messages = [{"role": "user", "content": content}]

    completion = client.chat.completions.create(
        model="llama-3.2-11b-vision-preview",
        messages=messages,
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    return completion.choices[0].message.content

# Example usage
if __name__ == "__main__":
    file_path = "/content/sample_data/PS_2.1_011_1075_05_Relevante Eingangsgrößen für P3+ Anmeldepackage erzeugen.pdf"  # Path to your file (image or PDF)
    text_prompt = "Analyze this file and provide insights."
    result = analyze_file(file_path, text_prompt)
    print("AI Analysis:", result)

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.2-11b-vision-preview` in organization `org_01jk7tcn4be9h8bya1x2691xaa` service tier `on_demand` on tokens per minute (TPM): Limit 7000, Requested 7652, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

# Using Split and Read Approach

In [1]:
!pip install fitz PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 38.2 MB/s eta 0:00:00


In [4]:
import fitz  # PyMuPDF
import os

def split_pdf(input_pdf, output_folder):
    """
    Splits a PDF file into individual pages.

    :param input_pdf: Path to the input PDF file.
    :param output_folder: Folder where the split pages will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    doc = fitz.open(input_pdf)
    num_pages = len(doc)  # Get the number of pages before closing the document
    for page_num in range(num_pages):
        output_pdf_path = os.path.join(output_folder, f"page_{page_num + 1}.pdf")
        new_doc = fitz.open()
        new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
        new_doc.save(output_pdf_path)
        new_doc.close()

    doc.close()
    print(f"PDF split into {num_pages} pages and saved in {output_folder}")  # Use num_pages here

def zoom_image_in_pdf(input_pdf, output_pdf, zoom_factor):
    """
    Zooms images in a PDF file.

    :param input_pdf: Path to the input PDF file.
    :param output_pdf: Path to the output PDF file with zoomed images.
    :param zoom_factor: Scaling factor for zooming.
    """
    doc = fitz.open(input_pdf)
    mat = fitz.Matrix(zoom_factor, zoom_factor)

    new_doc = fitz.open()

    for page in doc:
        pix = page.get_pixmap(matrix=mat)
        img_page = new_doc.new_page(width=pix.width, height=pix.height)
        img_page.insert_image(img_page.rect, pixmap=pix)

    new_doc.save(output_pdf)
    new_doc.close()
    doc.close()
    print(f"Zoomed PDF saved as {output_pdf}")

# Example usage
split_pdf("/content/zoomed_output.pdf", "split_pages")
# zoom_image_in_pdf("/content/sample_data/PS_2.1_011_1075_05_Relevante Eingangsgrößen für P3+ Anmeldepackage erzeugen.pdf", "zoomed_output.pdf", 6.0)

PDF split into 11 pages and saved in split_pages
